# Backbone Preprocessing

This notebook encodes frames into feature vectors given a pretrained CNN.

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import keras

## Define the feature extractor model

In [2]:
class FeatureExtractor(tf.keras.Model):
    def __init__(self, model_name,layer=None):
        """
            Initialize Feature extractor with a pretrained CNN model

            Args:
                model_name: name of the pretrained CNN model ["resnet", "inception_resnet"]
        """
        super(FeatureExtractor, self).__init__()
        if model_name == "resnet":
            from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
            if not layer:
                self.model = ResNet50V2(include_top=False, weights='imagenet', pooling='avg')
            else:
                resnet = ResNet50V2(include_top=False, weights='imagenet', pooling='avg')
                self.model = keras.Model(inputs=resnet.input,outputs=keras.layers.GlobalAveragePooling2D()(resnet.get_layer(layer).output))
            self.model_input_size = (224, 224)
        elif model_name == "inception_resnet":
            from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
            self.model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg')
            self.model_input_size = (299, 299)
        else:
            raise NameError('Invalid pretrained model name - must be one of ["resnet", "inception_resnet"]')
        
        self.preprocess_input = preprocess_input
        self.model.trainable = False

    def call(self, inputs):
        """
            Call the pretrained CNN model to predict the features for a given input image

            Args:
                inputs: input image tensor
        """
        # Resize inputs to the expected input size
        inputs = inputs*255
        inputs = tf.image.resize(inputs, self.model_input_size)
        inputs = inputs[tf.newaxis, :]
        preprocessed_input = self.preprocess_input(inputs)
        return self.model.predict(preprocessed_input).ravel()

We can visualize here the backbones architectures:

In [4]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

resnet =  ResNet50V2(include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3))
resnet.summary()

Model: "resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 23,519,360 (89.72 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [5]:
model = keras.Model(inputs=resnet.input,outputs=keras.layers.GlobalAveragePooling2D()(resnet.get_layer('conv5_block2_1_relu').output))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 15,673,856 (59.79 MB)

 Trainable params: 15,639,680 (59.66 MB)

 Non-trainable params: 34,176 (133.50 KB)

# Define input and output folders

In [6]:
# Dataset input and output folders
image_dataset_path = '../data/processed/ImageDatasetRGB'
dataset_path = '../data/processed/512_Dataset'

In [7]:
videos = sorted(os.listdir(os.path.join(image_dataset_path, 'features')))
labels = sorted(os.listdir(os.path.join(image_dataset_path, 'labels')))

if '.ipynb_checkpoints' in videos:
    videos.remove('.ipynb_checkpoints')

if '.ipynb_checkpoints' in labels:
    labels.remove('.ipynb_checkpoints')
    
print(videos)
print(labels)

['1', '2', '3', '4', '5', 'Animal61980', 'Animal62418', 'HD_ChR2_480A', 'HD_ChR2_510A', 'HD_ChR2_586A', 'HD_YFP_037A', 'HD_YFP_443A', 'HD_YFP_463A', 'WT_ChR2_087A', 'WT_ChR2_400A', 'WT_ChR2_425A', 'WT_ChR2_635A', 'WT_ChR2_654A', 'WT_YFP_154A', 'WT_YFP_435A', 'WT_YFP_535A', 'WT_YFP_602A', 'WT_YFP_741A', 'WT_YFP_792']
['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', 'Animal61980.csv', 'Animal62418.csv', 'HD_ChR2_480A.csv', 'HD_ChR2_510A.csv', 'HD_ChR2_586A.csv', 'HD_YFP_037A.csv', 'HD_YFP_443A.csv', 'HD_YFP_463A.csv', 'WT_ChR2_087A.csv', 'WT_ChR2_400A.csv', 'WT_ChR2_425A.csv', 'WT_ChR2_635A.csv', 'WT_ChR2_654A.csv', 'WT_YFP_154A.csv', 'WT_YFP_435A.csv', 'WT_YFP_535A.csv', 'WT_YFP_602A.csv', 'WT_YFP_741A.csv', 'WT_YFP_792.csv']


In [8]:
split = pd.read_csv(os.path.join('../data/processed/split.csv'))

## ResNet Feature Extraction

In [9]:
feature_extractor = FeatureExtractor('resnet', layer='conv5_block2_1_relu')

In [10]:
resnet_path = os.path.join(dataset_path, 'resnet')
os.mkdir(resnet_path)

In [12]:
os.mkdir(os.path.join(resnet_path, 'train'))
os.mkdir(os.path.join(resnet_path, 'train', 'features'))
os.mkdir(os.path.join(resnet_path, 'train', 'labels'))
os.mkdir(os.path.join(resnet_path, 'test'))
os.mkdir(os.path.join(resnet_path, 'test', 'features'))
os.mkdir(os.path.join(resnet_path, 'test', 'labels'))
os.mkdir(os.path.join(resnet_path, 'validation'))
os.mkdir(os.path.join(resnet_path, 'validation', 'features'))
os.mkdir(os.path.join(resnet_path, 'validation', 'labels'))

### Direct extraction

In [ ]:
for video, label in zip(videos, labels):

    print(' -------- Video ', video, ' --------')
    features = []
    for frame_name in sorted(os.listdir(os.path.join(image_dataset_path, 'features', video))):
        if frame_name == '.ipynb_checkpoints':
            continue
        frame = tf.io.read_file(os.path.join(image_dataset_path, 'features', video, frame_name))
        frame = tf.image.decode_image(frame, channels=3)
        frame = tf.image.convert_image_dtype(frame, tf.float32)
        features.append(feature_extractor(frame))

    video_features = np.array(features)
    video_labels = pd.read_csv(os.path.join(image_dataset_path, 'labels', label))

    if split[split['name'] == video]['set'].values == 'train':
        output_folder = os.path.join(resnet_path, 'train')
    elif split[split['name'] == video]['set'].values == 'validation':
        output_folder = os.path.join(resnet_path, 'validation')
    else:
        output_folder = os.path.join(resnet_path, 'test')
    np.save(os.path.join(output_folder, 'features', video + '.npy'), video_features)
    video_labels.to_csv(os.path.join(output_folder, 'labels', video + '.csv'), index=None)
    print('.', end='')
    
print('\n')

 -------- Video  1  --------
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 829ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━

In [ ]:
#Check if the last element was writted correctly
last =  np.load("../data/processed/Dataset/resnet/test/features/WT_YFP_741A.npy")
print(last.shape)
#Check if the last element was writted correctly
first =  np.load("../data/processed/Dataset/resnet/test/features/WT_ChR2_654A.npy")
first.shape

### Extraction with counter
This extraction method is recommended for those computers who are more likely to give a dead kernel due to lower computing capacities

In [11]:
video_count = 0
frame_count = 0  

In [9]:
for video, label in zip(videos[video_count:], labels[video_count:]):
    features = []
    print("Video ", video_count)
    
    if frame_count != 0:
        
        frames = sorted(os.listdir(os.path.join(image_dataset_path, 'features', video)))
        
        for frame_name in frames[frame_count:]:
            print("Frame ", frame_count, " from video ", video_count)
            frame = tf.io.read_file(os.path.join(image_dataset_path, 'features', video, frame_name))
            frame = tf.image.decode_image(frame, channels=3)
            frame = tf.image.convert_image_dtype(frame, tf.float32)
            features.append(feature_extractor(frame))
            frame_count += 1
        
    else:
        for frame_name in sorted(os.listdir(os.path.join(image_dataset_path, 'features', video))):
            print("Frame ", frame_count, " from video ", video_count)
            frame = tf.io.read_file(os.path.join(image_dataset_path, 'features', video, frame_name))
            frame = tf.image.decode_image(frame, channels=3)
            frame = tf.image.convert_image_dtype(frame, tf.float32)
            features.append(feature_extractor(frame))
            frame_count += 1
        
    video_count += 1
    frame_count = 0

    video_features = np.array(features)
    video_labels = pd.read_csv(os.path.join(image_dataset_path, 'labels', label))

    if split[split['name'] == video]['set'].values == 'train':
        output_folder = os.path.join(resnet_path, 'train')
    else:
        output_folder = os.path.join(resnet_path, 'test')
    np.save(os.path.join(output_folder, 'features', video + '.npy'), video_features)
    video_labels.to_csv(os.path.join(output_folder, 'labels', video + '.csv'), index=None)
    
    print('.', end='')
    
print('\n')


Video  14
Frame  3376  from video  14
1/1 [==============================] - 3s 3s/step
Frame  3377  from video  14
1/1 [==============================] - 0s 23ms/step
Frame  3378  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3379  from video  14
1/1 [==============================] - 0s 23ms/step
Frame  3380  from video  14
1/1 [==============================] - 0s 23ms/step
Frame  3381  from video  14
1/1 [==============================] - 0s 23ms/step
Frame  3382  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3383  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3384  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3385  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3386  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3387  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3388  from video  14
1/1 

1/1 [==============================] - 0s 32ms/step
Frame  3479  from video  14
1/1 [==============================] - 0s 28ms/step
Frame  3480  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3481  from video  14
1/1 [==============================] - 0s 19ms/step
Frame  3482  from video  14
1/1 [==============================] - 0s 19ms/step
Frame  3483  from video  14
1/1 [==============================] - 0s 19ms/step
Frame  3484  from video  14
1/1 [==============================] - 0s 28ms/step
Frame  3485  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3486  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3487  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3488  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3489  from video  14
1/1 [==============================] - 0s 28ms/step
Frame  3490  from video  14
1/1 [==============================] - 0

1/1 [==============================] - 0s 30ms/step
Frame  3683  from video  14
1/1 [==============================] - 0s 30ms/step
Frame  3684  from video  14
1/1 [==============================] - 0s 28ms/step
Frame  3685  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3686  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3687  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3688  from video  14
1/1 [==============================] - 0s 29ms/step
Frame  3689  from video  14
1/1 [==============================] - 0s 21ms/step
Frame  3690  from video  14
1/1 [==============================] - 0s 29ms/step
Frame  3691  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3692  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3693  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3694  from video  14
1/1 [==============================] - 0

1/1 [==============================] - 0s 27ms/step
Frame  3887  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3888  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3889  from video  14
1/1 [==============================] - 0s 26ms/step
Frame  3890  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3891  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3892  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3893  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3894  from video  14
1/1 [==============================] - 0s 20ms/step
Frame  3895  from video  14
1/1 [==============================] - 0s 29ms/step
Frame  3896  from video  14
1/1 [==============================] - 0s 27ms/step
Frame  3897  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  3898  from video  14
1/1 [==============================] - 0

1/1 [==============================] - 0s 31ms/step
Frame  4091  from video  14
1/1 [==============================] - 0s 25ms/step
Frame  4092  from video  14
1/1 [==============================] - 0s 26ms/step
Frame  4093  from video  14
1/1 [==============================] - 0s 29ms/step
Frame  4094  from video  14
1/1 [==============================] - 0s 30ms/step
Frame  4095  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  4096  from video  14
1/1 [==============================] - 0s 25ms/step
Frame  4097  from video  14
1/1 [==============================] - 0s 26ms/step
Frame  4098  from video  14
1/1 [==============================] - 0s 23ms/step
Frame  4099  from video  14
1/1 [==============================] - 0s 22ms/step
Frame  4100  from video  14
1/1 [==============================] - 0s 30ms/step
Frame  4101  from video  14
1/1 [==============================] - 0s 26ms/step
Frame  4102  from video  14
1/1 [==============================] - 0

...

1/1 [==============================] - 0s 20ms/step
Frame  5619  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  5620  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5621  from video  16
1/1 [==============================] - 0s 22ms/step
Frame  5622  from video  16
1/1 [==============================] - 0s 24ms/step
Frame  5623  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5624  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5625  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  5626  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  5627  from video  16
1/1 [==============================] - 0s 22ms/step
Frame  5628  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5629  from video  16
1/1 [==============================] - 0s 22ms/step
Frame  5630  from video  16
1/1 [==============================] - 0

1/1 [==============================] - 0s 19ms/step
Frame  5823  from video  16
1/1 [==============================] - 0s 21ms/step
Frame  5824  from video  16
1/1 [==============================] - 0s 26ms/step
Frame  5825  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5826  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  5827  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5828  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  5829  from video  16
1/1 [==============================] - 0s 21ms/step
Frame  5830  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  5831  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  5832  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  5833  from video  16
1/1 [==============================] - 0s 21ms/step
Frame  5834  from video  16
1/1 [==============================] - 0

1/1 [==============================] - 0s 19ms/step
Frame  6027  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6028  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6029  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6030  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  6031  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  6032  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  6033  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6034  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6035  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6036  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6037  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6038  from video  16
1/1 [==============================] - 0

1/1 [==============================] - 0s 19ms/step
Frame  6231  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6232  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  6233  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6234  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6235  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6236  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6237  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6238  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6239  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6240  from video  16
1/1 [==============================] - 0s 18ms/step
Frame  6241  from video  16
1/1 [==============================] - 0s 28ms/step
Frame  6242  from video  16
1/1 [==============================] - 0

1/1 [==============================] - 0s 21ms/step
Frame  6435  from video  16
1/1 [==============================] - 0s 21ms/step
Frame  6436  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6437  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6438  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6439  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6440  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6441  from video  16
1/1 [==============================] - 0s 31ms/step
Frame  6442  from video  16
1/1 [==============================] - 0s 29ms/step
Frame  6443  from video  16
1/1 [==============================] - 0s 19ms/step
Frame  6444  from video  16
1/1 [==============================] - 0s 20ms/step
Frame  6445  from video  16
1/1 [==============================] - 0s 26ms/step
Frame  6446  from video  16
1/1 [==============================] - 0

## InceptionResNet Feature Extraction

In [6]:
feature_extractor = FeatureExtractor('inception_resnet')

In [7]:
inception_resnet_path = os.path.join(dataset_path, 'inception_resnet')
os.mkdir(inception_resnet_path)

In [8]:
os.mkdir(os.path.join(inception_resnet_path, 'train'))
os.mkdir(os.path.join(inception_resnet_path, 'train', 'features'))
os.mkdir(os.path.join(inception_resnet_path, 'train', 'labels'))
os.mkdir(os.path.join(inception_resnet_path, 'test'))
os.mkdir(os.path.join(inception_resnet_path, 'test', 'features'))
os.mkdir(os.path.join(inception_resnet_path, 'test', 'labels'))

In [ ]:
for video, label in zip(videos, labels):
    features = []
    for frame_name in sorted(os.listdir(os.path.join(image_dataset_path, 'features', video))):
        frame = tf.io.read_file(os.path.join(image_dataset_path, 'features', video, frame_name))
        frame = tf.image.decode_image(frame, channels=3)
        frame = tf.image.convert_image_dtype(frame, tf.float32)
        features.append(feature_extractor(frame))

    video_features = np.array(features)
    video_labels = pd.read_csv(os.path.join(image_dataset_path, 'labels', label))

    if split[split['name'] == video]['set'].values == 'train':
        output_folder = os.path.join(inception_resnet_path, 'train')
    else:
        output_folder = os.path.join(inception_resnet_path, 'test')
    np.save(os.path.join(output_folder, 'features', video + '.npy'), video_features)
    video_labels.to_csv(os.path.join(output_folder, 'labels', video + '.csv'), index=None)
    print('.', end='')
    
print('\n')